In [1]:
import os
import numpy as np
import yaml

# PyTorch and torchvision imports
import torch
from torch.utils.data import DataLoader 
import torchvision.transforms as T

# Custom models and datasets
from dataset.video_dataset import AdImageSequenceDataset
from models.features_extractors import EfficientNet_feature_B5

# Custom utils
from utils.utils import CosineLoss, Gaussian

# Evaluation
from torchmetrics.classification import BinaryAUROC

import onnxruntime as ort

# Clear CUDA cache and enable CuDNN for better performance
torch.cuda.empty_cache()
torch.backends.cudnn.enabled = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/workspaces/XLIM_Thesis/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load configuration from YAML file
with open('config_inference.yaml', 'r') as file:
    config = yaml.safe_load(file)


In [3]:
# Define image preprocessing transformations for EfficientNet
preprocessing_efficientnet = T.Compose([
    T.Resize((config['general_config']['image_size'], 
              config['general_config']['image_size']), 
             interpolation=T.InterpolationMode.BICUBIC),
    T.ToTensor(),
    T.Normalize(mean=[0.4850, 0.4560, 0.4060],
                std=[0.2290, 0.2240, 0.2250])
])

# Define image preprocessing transformations for labels
preprocessing_labels = T.Compose([
    T.Resize((config['general_config']['feature_size'], config['general_config']['feature_size'])),
    T.ToTensor()
])

# Create validation dataset
dataset_val = AdImageSequenceDataset(
    os.path.join(config['general_config']['data_path']),
    preprocessing_efficientnet,
    preprocessing_labels
)

dataloader_val = DataLoader(
    dataset_val,
    batch_size=1,
    shuffle=False,
    num_workers=8
)

print(len(dataloader_val))
gs_filter = Gaussian(device = 'cuda')

2996


In [4]:
onnx_model_path = config['general_config']['weight_path_VMTAD_onnx']
VMTAD_session = ort.InferenceSession(onnx_model_path, providers=['CUDAExecutionProvider'])
input_names = [input.name for input in VMTAD_session.get_inputs()]

feature_extraction  = EfficientNet_feature_B5(config)
feature_extraction.eval() 

print("Models load")

Unexpected keys (bn2.num_batches_tracked, bn2.bias, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


Models load


## ONNX Version VMTAD_B5

In [5]:


# Evaluation setup 
eval_seg = np.zeros((len(dataset_val),config['general_config']['feature_size'],config['general_config']['feature_size']))
labels_seg = np.zeros((len(dataset_val),config['general_config']['feature_size'],config['general_config']['feature_size']))
eval_det = np.zeros(len(dataset_val))
labels_det = np.zeros(len(dataset_val))

# Loss setup
loss_visu = CosineLoss()
auroc_inst = BinaryAUROC().cuda()




# loop setup
folder_indice = None

# evaluation loop
for k,(_, folder, images, labels) in enumerate(dataloader_val):

    images_cuda = images.to('cuda')
    labels_np = labels[0,1].numpy().astype(int)
    with torch.no_grad():
            
        input_features=feature_extraction(images_cuda).detach().cpu().numpy().astype(np.float32)
        outputs = VMTAD_session.run(None, {'input.1': input_features})
        output = outputs[0]
        src = outputs[1]
        visu_loss = loss_visu(torch.from_numpy(output).to('cuda'),torch.from_numpy(src).to('cuda'))
        anomaly_map_smooth = gs_filter(visu_loss).detach().cpu().numpy()
        
        eval_seg[k] = anomaly_map_smooth
        labels_seg[k] = labels_np
                
        eval_det[k] = np.max(anomaly_map_smooth)
        labels_det[k] = np.max(labels_np)
        
        
# Print evaluation results        
print("AUROC Det: ",auroc_inst(torch.from_numpy(eval_det).to('cuda'), torch.from_numpy(labels_det).to('cuda')))
print("AUROC Seg: ",auroc_inst(torch.from_numpy(eval_seg).to('cuda'), torch.from_numpy(labels_seg).to('cuda')))

AUROC Det:  tensor(0.9732, device='cuda:0')
AUROC Seg:  tensor(0.9970, device='cuda:0')
